In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

/home/queimadas/.conda/envs/py3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df_conhecimento = pd.read_csv('conhecimento_filtrado.csv')

In [4]:
df_full = pd.read_csv('dados_sem_auditoria.csv')
id_cena = 'LS82260702018259'

In [5]:
df = df_full[df_full['id_cena']==id_cena].copy()

In [6]:
df.shape

(3618, 13)

In [7]:
df_conhecimento = df_conhecimento[df_conhecimento.id_cena != id_cena]

In [8]:
df_conhecimento.shape

(81747, 13)

In [9]:
colunas_stats = ['ndvi', 'nbrl', 'dif_ndvi', 'dif_dnbrl', 'medianb2', 'medianb3', 'medianb4', 'medianb5', 'medianb6', 'medianb7']

In [10]:
print('----------------------')
print('Criando classificador RF')
rf_clas = RandomForestClassifier(n_jobs=8, n_estimators=100, max_features=5, random_state=0, max_depth=3, min_samples_split=5)
rf_clas.fit(df_conhecimento[colunas_stats], df_conhecimento.verifica)
print('----------------------\n\n\n')

----------------------
Criando classificador RF
----------------------





In [11]:
print('----------------------')
print('Criando classificador RF')
rf_clas = RandomForestClassifier(n_jobs=8, n_estimators=100, max_features=5, random_state=0, max_depth=3, min_samples_split=5)
rf_clas.fit(df_conhecimento[colunas_stats], df_conhecimento.verifica)
print('----------------------\n\n\n')


----------------------
Criando classificador RF
----------------------





In [12]:
print('----------------------')
print('Criando rank_RF colunas')
df_rank_rf = pd.DataFrame(sorted(list(zip(colunas_stats,rf_clas.feature_importances_)), key=lambda x: x[1], reverse=True))
print('----------------------\n\n\n')

----------------------
Criando rank_RF colunas
----------------------





In [13]:
print('----------------------')
print('Divide colunas em 2 grupos - RF')
dados = df_rank_rf[[1]].values
kmeans = KMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300, n_jobs=8)
kmeans = kmeans.fit(dados)
labels = kmeans.predict(dados)
df_rank_rf['labels'] = labels
print('----------------------\n\n\n')


----------------------
Divide colunas em 2 grupos - RF
----------------------





In [14]:
df_rank_rf

,0,1,labels
0,medianb5,0.429695,1
1,nbrl,0.242108,1
2,dif_ndvi,0.093210,0
3,medianb6,0.092482,0
4,medianb3,0.067609,0
5,ndvi,0.046316,0
6,medianb2,0.010345,0
7,medianb7,0.008553,0
8,dif_dnbrl,0.008404,0
9,medianb4,0.001280,0


In [15]:
print('----------------------')
print('Utiliza somente o grupo colunas mais relevante RF')
label_grupo_bom = df_rank_rf[df_rank_rf[1]==df_rank_rf[1].max()].labels.values[0]
colunas_usadas_rf = df_rank_rf[df_rank_rf['labels']==label_grupo_bom][0]
print (colunas_usadas_rf)
print('----------------------\n\n\n')


----------------------
Utiliza somente o grupo colunas mais relevante RF
0    medianb5
1        nbrl
Name: 0, dtype: object
----------------------





In [16]:

print('----------------------')
print('RF - Classifica dado usando as melhores colunas')
rf_clas = RandomForestClassifier(n_jobs=8, n_estimators=100, max_features=len(colunas_usadas_rf), random_state=0, max_depth=3, min_samples_split=5)
rf_clas.fit(df_conhecimento[colunas_usadas_rf], df_conhecimento.verifica)
df['rf'] = rf_clas.predict(df[colunas_usadas_rf])
df['rf_1'] = rf_clas.predict_proba(df[colunas_usadas_rf])[:, 0]
print('----------------------\n\n\n')


----------------------
RF - Classifica dado usando as melhores colunas
----------------------





In [17]:
print('----------------------')
print('Classificando com NN')
nn_clas = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.00010, max_iter=200)
nn_clas.fit(df_conhecimento[colunas_usadas_rf], df_conhecimento.verifica)
df['nn'] = nn_clas.predict(df[colunas_usadas_rf])
df['nn_1'] = nn_clas.predict_proba(df[colunas_usadas_rf])[:, 0]
print('----------------------\n\n\n')


----------------------
Classificando com NN
----------------------





In [18]:
print('----------------------')
print('Normaliza dados RF e NN')
rf_max = df.rf_1.max()
rf_min = df.rf_1.min()

nn_max = df.nn_1.max()
nn_min = df.nn_1.min()

df['rf_1'] = (df.rf_1 - rf_min)/(rf_max-rf_min)
df['nn_1'] = (df.nn_1 - nn_min)/(nn_max-nn_min)  # NN normalizado
print('----------------------\n\n\n')



----------------------
Normaliza dados RF e NN
----------------------





In [19]:

print('----------------------')
print('Cria avaliacao usando (RF+NN)/2 - Arredondado para 3 casas decimais')
df['avaliacao'] = (df["rf_1"] + df["nn_1"]) / 2
df['avaliacao'] = df['avaliacao'].apply(lambda x: round(x, 4))
df = df.sort_values('avaliacao', ascending=False).reset_index(drop=True)
print('----------------------\n\n\n')


----------------------
Cria avaliacao usando (RF+NN)/2 - Arredondado para 3 casas decimais
----------------------





In [20]:

print('----------------------')
df_saida = df[df['avaliacao'] > 0.05]
print('Linhas sem nuvem > 0.05', len(df[df['avaliacao'] > 0.05]))
print('----------------------\n\n\n')


----------------------
Linhas sem nuvem > 0.05 398
----------------------





<h1>TODO </h1>
<pre>
- Gerar uma amostra de 1000
- Pegar esta amostra e remove-la do conhecimento
- Limpar tag verifica
- Bateria de teste com conhecimentos diferentes(todos conhecimentos sem a amostra atual avaliada):
-- Conhecimento completo e TODAS colunas
-- Conhecimento completo e colunas selecionadas
-- Conhecimento gerado com AAS(amostra aleatoria simples) e TODAS colunas
-- Conhecimento gerado com AAS(amostra aleatoria simples) e colunas selecionadas
-- Conhecimento trimestre historico
- Gerar matriz de confusão para cada teste
</pre>